In [30]:
import requests
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains.summarize import load_summarize_chain
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.chains import RetrievalQA, TransformChain, SequentialChain, LLMChain

In [4]:
url = "https://www.cs.virginia.edu/~evans/greatworks/diffie.pdf"
response = requests.get(url)

In [5]:
with open("../files/public_key_crytptography.pdf", "wb") as f:
    f.write(response.content)

loader = PyPDFLoader("../files/public_key_crytptography.pdf")
data = loader.load()

In [6]:
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(data, embeddings)

In [7]:
len(data)

18

In [12]:
llm_gpt = OpenAI()
chain_summarize = load_summarize_chain(llm_gpt, chain_type="map_reduce")
chain_summarize.run(data)

' \n\nPublic-key cryptography was discovered in 1975 and revolutionized communication security by allowing for secure communication between many subscribers. It was developed by Roger Needham and Martin Hellman and uses one-way and trap-door functions to protect computer passwords. The RSA cryptosystem, using large prime numbers, is one of its implementations. The development of public-key cryptography faced challenges, but it has been used in various applications, including military identification systems. It has also been commercialized by companies such as RSA Data Security and Cylink Corporation. The National Security Agency has also conducted feasibility studies for a new secure phone system using public-key technology. Research in this field is ongoing, with a focus on improving factoring and finding prime numbers. The success of public-key cryptography has delayed the introduction of new systems, and it is widely accepted and used in various secure devices. Mathematician Whitfie

In [14]:
template = """
Escribe un resumen bien chido del siguient rollo:

{text}

RESUMEN CORTO CON SLANG MEXICANO:
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["text"],
)

In [15]:
chain_summarizer_w_slang = load_summarize_chain(
    llm_gpt, chain_type="stuff", prompt=prompt, verbose=True
)

chain_summarizer_w_slang.run(data[:2])



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Escribe un resumen bien chido del siguient rollo:

The First Ten Years of Public-Key 
Cryptography 
WH lTFl ELD DI FFlE 
Invited Paper 
Public-key cryptosystems separate the capacities for encryption 
and decryption so that 7) many people can encrypt messages in 
such a way that only one person can read them, or 2) one person 
can encrypt messages in such a way that many people can read 
them. This separation allows important improvements in the man- 
agement of cryptographic keys and makes it possible to ‘sign’ a 
purely digital message. 
Public key cryptography was discovered in the Spring of 1975 
and has followed a surprising course. Although diverse systems 
were proposed early on, the ones that appear both practical and 
secure today are all very closely related and the search for new and 
different ones has met with little success. Despite this reliance on 
a limited mathem

'\nEl broncón de la criptografía de llave pública revolucionó la seguridad de las comunicaciones al permitir a muchas personas encriptar mensajes de manera que solo una persona pueda leerlos o viceversa. Se descubrió en 1975 y ha tenido un gran impacto tanto en la práctica como en la teoría de la seguridad de las comunicaciones. Antes de esto, se creía que era imposible resolver el problema de la distribución de llaves y de las firmas digitales, pero gracias a la utilización de una función de llave trap-door (conocida como función de un solo sentido), se pudo solucionar ambos problemas al mismo tiempo. Esto permitió una gestión más fácil de las llaves y la realización de transacciones de mayor valor en redes de comunicación. Aunque existen otras soluciones, la criptografía de llave pública es la más eficiente y evita la intervención de terceros en las comunicaciones. Todo esto se logró gracias al trabajo de Whitfield Diffie y Martin Hellman en 1975 en la Universidad de Stanford.'

In [17]:
chain_qa = RetrievalQA.from_chain_type(
    llm=llm_gpt,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 2}),
)

In [18]:
chain_qa.run("Cual es la relevancia de la criptografia de llave publica?")

' La criptografía de llave pública es un método de encriptación en el que se utiliza un par de llaves, una pública y otra privada, para cifrar y descifrar información. Es especialmente relevante en la seguridad de comunicaciones en línea, ya que permite a los usuarios comunicarse de forma segura sin necesidad de compartir una llave secreta previamente acordada. Además, la criptografía de llave pública también se utiliza en la autenticación y firma digital, lo que la hace esencial para la seguridad en transacciones y operaciones en línea.'

## Foundational Chains

In [19]:
import re

In [21]:
def clean_text(inputs: dict) -> dict:
    text = inputs["text"]

    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001f64f"
        "\U0001F300-\U0001F5FF"
        "\U0001F680-\U0001F6FF"
        "\U0001F1E0-\U0001F1FF"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    text = emoji_pattern.sub(r"", text)

    url_pattern = re.compile(r"https?://\S+|www\.\S+")
    text = url_pattern.sub(r"", text)
    return {"clean_text": text}

In [24]:
chain_cleaner = TransformChain(
    input_variables=["text"], output_variables=["clean_text"], transform=clean_text
)

In [25]:
chain_cleaner.run("Check this page https://x.com/home 🔥")

'Check this page  '

In [28]:
template = """
Parafrasea este texto:

{clean_text}

Con acento y estilo {style}

Parafraseado:
"""

In [29]:
prompt = PromptTemplate(input_variables=["clean_text", "style"], template=template)

In [31]:
chain_change_style = LLMChain(llm=llm_gpt, prompt=prompt, output_key="final_text")

In [32]:
chain_sequential = SequentialChain(
    chains=[chain_cleaner, chain_change_style],
    input_variables=["text", "style"],
    output_variables=["final_text"],
)

In [33]:
input_text = """
El Salvador 🇸🇻 (en náhuat, Kuskatan), oficialmente República de El Salvador www.elsalvador.com, 
es un país soberano de América Central ubicado en el litoral del océano 
Pacífico🌊, con una extensión territorial de 21 041 km².10​ En el año 2021 
contaba con una población estimada en 6 803 023 habitantes según una estimación 
realizada por la Dirección General de Estadísticas y Censos en compañía de la CEPAL,
11​ y poseía una densidad demográfica aproximada de 323 hab./km², lo que lo 
convierte en el país más densamente poblado del continente americano, 
sin incluir algunas islas en el mar Caribe. Su clima es cálido tropical🌴, 
pero debido al contraste geográfico el clima☔️ puede variar.
"""

In [39]:
chain_sequential({"text": input_text, "style": "nahuat"})

{'text': '\nEl Salvador 🇸🇻 (en náhuat, Kuskatan), oficialmente República de El Salvador www.elsalvador.com, \nes un país soberano de América Central ubicado en el litoral del océano \nPacífico🌊, con una extensión territorial de 21 041 km².10\u200b En el año 2021 \ncontaba con una población estimada en 6 803 023 habitantes según una estimación \nrealizada por la Dirección General de Estadísticas y Censos en compañía de la CEPAL,\n11\u200b y poseía una densidad demográfica aproximada de 323 hab./km², lo que lo \nconvierte en el país más densamente poblado del continente americano, \nsin incluir algunas islas en el mar Caribe. Su clima es cálido tropical🌴, \npero debido al contraste geográfico el clima☔️ puede variar.\n',
 'style': 'nahuat',
 'final_text': '\nSālākhuān (in tetl Kuskatan), tlahtohco República de Sālākhuān, tlācatlampa ye kōzkatlāntli īpan īlhuicac okeānōtlācatl, ye īpan īpantlācatl ye ōmpatlācatl kālīhuatl īpan 21 041 km².10\u200b Īka 2021 ye tlahtohco īpan 6 803 023 ininē